In [1]:
import pandas

In [14]:
class table():

    def __init__(self, file_name):
        self.commands = self.getCommands(file_name)
        self.contents = self.makeTable(self.commands)
        self.table = pandas.DataFrame(self.contents)

    def getCommands(self, file_name):
        commands = []
        with open(file_name) as file:
            for line in file:
                commands.append(line.strip())
        return commands
    
    def findOperation(self, line):
        if line.find("is-a")!=-1:
            return ("is-a")
        elif line.find("has-row")!=-1:
            return ("has-row")
        elif line.find("has-slot")!=-1:
            return ("has-slot")
        elif line.find("starts-at")!=-1:
            return ("starts-at")
        elif line.find("ends-at")!=-1:
            return ("ends-at") 

    def getArguments(self, line):
        return [ entry.strip() for entry in line.split(self.findOperation(line)) ]
    
    def getColumns(self):
        header = ["Row"] + [str(time) for time in range(8,17+1)]
        columns = dict( [column, None] for column in header )
        return columns

    def makeRows(self, commands):
        rows = []
        for line in commands:
            if self.findOperation(line)=="has-row":
                rows.append(self.getArguments(line)[1])
        return rows

    def getTime(self, commands):
        slots = []
        for line in commands:
            if self.findOperation(line)=="starts-at":
                slots.append(self.getArguments(line)[0]) 
        slots = dict([slot, None] for slot in slots)

        for slot in slots:
            related = [line for line in commands if self.getArguments(line)[0]==slot]
            start = [self.getArguments(line)[1] for line in related if self.findOperation(line)=="starts-at"]
            end = [self.getArguments(line)[1] for line in related if self.findOperation(line)=="ends-at"]
            start = int(start[0].split(':')[0])
            end = int(end[0].split(':')[0])

            slots[slot] = [str(time) for time in range(start, end+1)]   
        return slots

    def getSlots(self, commands):
        slots = dict([ [row, list()] for row in self.makeRows(commands) ])
        for line in commands:
            if self.findOperation(line)=="has-slot":
                row, slot = self.getArguments(line)
                slots[row].append([slot, self.getTime(commands)[slot]])
        return slots

    def makeTable(self, commands):
        slots = self.getSlots(commands)
        rows = []
        for row in slots.keys():
            data = self.getColumns()
            data["Row"] = row
            for slot in slots[row]:
                for time in slot[1]:
                    data[time] = slot[0]
            rows.append(data)
        return rows

    def canOverlap(self, row1, row2):
        flag=True
        for row in self.contents:
            if row["Row"]==row1:
                rowA = row
            if row["Row"]==row2:
                rowB = row
        for key in self.getColumns().keys():
            if key!="Row":
                if rowA[key]!=None and rowB[key]!=None:
                    flag=False
                    break
        return flag

    def reduce(self):
        clashes = []
        pairable = []
        rows = self.makeRows(self.commands)
        for row1 in rows:
            for row2 in rows:
                if row1!=row2:
                    if self.canOverlap(row1, row2):
                        if pairable.count(sorted([row1,row2]))==0:
                            pairable.append(sorted([row1,row2]))
                    else:
                        if clashes.count(sorted([row1,row2]))==0:
                            clashes.append(sorted([row1,row2]))
        return clashes, pairable

    def getTable(self):
        return self.table      

In [15]:
tab = table("table.txt")